# <center> R1.13 introduction à Python <br> TP4 - Nim, Wythoff et IA </center>
<center> 2023/2024 - Thibault Godin </center>
<center> IUT de Vannes, BUT Informatique </center>

***

Dans ce TP, on va voir une méthode d'intelligence artificielle classique, **l'apprentissage par renforcement**


On va présenter cette notion à l'aide d'une activité proposée par des collègues de la (Maison des Mathématiques et de l'Informatique)[https://mmi-lyon.fr/] (MMI) et du réseau Informatique Débranchée sur le jeu de _Nim_, puis on essaiera de l'appliquer au jeu de Withoff vu en JAVA en R1.01



- https://mmi-lyon.fr/?site_ressource_peda=jeu-de-nim-et-ia-avec-python
- https://mmi-lyon.fr/?site_ressource_peda=jeu-de-nim-et-ia

In [24]:
import random
import time

import numpy as np


import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 12)


# Jeu de Nim


## Mise en place


Le jeu de Nim se joue à 2 joueurs selon les règles suivantes : 
- On commence la partie avec un tas de $n$ allumettes
- Les deux joueurs jouent alternativement
- Seulement 1, 2, ou 3 allumette(s) peuvent être retirée(s) par un joueur à chaque tour
- La personne qui tire la dernière alumettre a gagné (version normale, on peut aussi dire "la personne qui ne peut plus jouer a perdu")

>**_question 1_** <br>
jouer (en silence) une ou deux parties avec un voisin

>**_question 2_**<br> 
lire (en diagonale) à quoi ressemblent les [stratégies gagnantes](https://interstices.info/jeux-de-nim/) de ce jeu 

>**_question 3_**<br> 
visualiser et comprendre l'apprentissage par renforcement proposé pour le jeu de Nim sur https://projet.liris.cnrs.fr/~mam/machine/




--------------------------



## Niammi : Nim Artificial Intelligence Maison des Mathématiques et de l'Informatique

Voici un programme Python (implémenté par Olivier Druet de la MMI) permettant de jouer au jeu de Nim contre un ordinateur _qui apprend au fur et à mieux jouer_. 

>**_question 4_**<br> 
lire et comprendre

# Wythoff


plan conseillé : 

1. affichage du plateau
2. alternance des joueurs et propositions des coups admissibles
3. condition de victoire
4. jeu humain v. humain
5. stockage de stratégie
6. jeu humain v. ordinateur (aléatoire)
8. règle de l'apprentissage de l'ordinateur (en cas de victoire ou de défaite
9. entrainement de l'ordinateur contre l'humain
10. entrainement de l'ordinateur contre le hasard
11. entrainement de l'ordinateur contre l'ordinateur

Vous pouvez bien sûr recycler du code Java de R1.01, ainsi que le code de Nim proposé plus haut (essayez d'être aussi clair dans votre code).

Le jeu de Wythoff étant plus complexe que celui de Nim, on fera des fonctions !

Pensez à documenter, commenter et illustrer votre TP.

**bonus** Une fois la machine entrainée, retrouver les positions gagnante à l'aide de la stratégie obtenue ; ou analyser la qualité de l'entrainement en fonction des paramètres de renforcement.

In [25]:
def tourJoueur(plateau,coJoueur):
    tour = 0
    distance = 1
    ligne = -len(plateau)
    colonne = -len(plateau[0])
    tourFait = False
    #pour le traitement du déplacement
    while ((tour != 1 and tour != 2 and tour != 3) or (coJoueur[1] < 0 or coJoueur[0] < 0 or coJoueur[1] < len(plateau) or coJoueur[0] or len(plateau[0])) and not(tourFait)):
        tour = 0
        print("(?) '1' pour déplacement à Gauche, '2' Diagonal, '3' en Bas (?)")
        tour = int(input())
        if (not (tour == 1 or tour == 2 or tour == 3)):
            print("[!] choix incorrect [!] ")
        else:

            print("(?) de combien ? (?)");
            distance = int(input()) 
            if (tour == 1): #gauche
                ligne = 0
                colonne = -distance
            elif (tour == 2): #diagonale
                ligne = -distance
                colonne = -distance
            elif (tour == 3): #bas
                ligne = -distance
                colonne = 0

            if (coJoueur[1] + ligne < 0 or coJoueur[0] + colonne < 0):
                print("[!] Trop grande distance [!]")
            elif (distance <= 0):
                print("[!] Trop petite distance [!]")
            else:
                tourFait = True
                plateau[coJoueur[1]][coJoueur[0]] = 0
                plateau[coJoueur[1] + ligne][coJoueur[0] + colonne] = 1
                coJoueur[1] = ligne
                coJoueur[0] = colonne


In [26]:
def start(taille):
    # créé un plateau de largeur et longeur taille rempli de 0
    plateau = [[0]*taille for _ in range(taille)]
    max = len(plateau)-2
    MIN = 1
    
    #placement aléatoire sur la première ligne
    if(random.randint(0,100) <= 50):
        plateau[random.randint(MIN,max)][len(plateau) - 1] = 1
    else: #placement aléatoire sur la dernière colonne
        plateau[len(plateau) -1][random.randint(MIN,max)] = 1
    return plateau


    
def tour(plateau, joueur1,coJoueur,ordi):
        joueur1 = not joueur1
        affichePlateau(plateau)
        if (plateau[0][0] != 1):
            if (joueur1):
                print("Au tour du joueur 1")
            elif (ordi):
                print("Au tour de l'ordinateur")
            else:
                print("Au tour du joueur 2 ")
            if (ordi and not(joueur1)):
                tourOrdi(plateau)
            else:
                tourJoueur(plateau,coJoueur)
        
        return joueur1
    

    
def transJoueur(plateau):
    result = [0,0]
    ligne = len(plateau) - 1
    while(ligne > 0):
        colonne = 0
        while(colonne < len(plateau[ligne])):
            if (plateau[ligne][colonne] == 1):
                result[1] = ligne
                result[0] = colonne
            colonne+=1
        ligne-=1
    return result

def affichePlateau(plateau):
    for ligne in range(len(plateau)-1,-1,-1):
        print(ligne,end ='')
        for colonne in range(0,len(plateau[ligne]),1):
            print(" | ", end='')
            if(plateau[ligne][colonne] == 1):
                print("o", end='')
            else:
                print(" ", end='')
        print(" |")
    print(" ", end='')
    for i in range(0,len(plateau),1):
        print("   ", end='')
        print(i,end='')
    print()
        

def update(plateau,joueur1,coJoueur,ordi):
    while(plateau[0][0] != 1):
        joueur1 = tour(plateau,joueur1,coJoueur,ordi)
        coJoueur = transJoueur(plateau)
    if(joueur1):
        print("joueur1 gagne")
    elif(ordi):
        print("l'ordi gagne (à récompensé)")
    else:
        print("joueur2 gagne")


In [27]:
#start
#mettre en False pour commencé avec le joueur 1 (oui je sais)
joueur1 = False
ordi = False
plateau = start(8)
coJoueur= transJoueur(plateau)
update(plateau,joueur1,coJoueur,ordi)

7 |   |   |   |   |   |   | o |   |
6 |   |   |   |   |   |   |   |   |
5 |   |   |   |   |   |   |   |   |
4 |   |   |   |   |   |   |   |   |
3 |   |   |   |   |   |   |   |   |
2 |   |   |   |   |   |   |   |   |
1 |   |   |   |   |   |   |   |   |
0 |   |   |   |   |   |   |   |   |
    0   1   2   3   4   5   6   7
Au tour du joueur 1
(?) '1' pour déplacement à Gauche, '2' Diagonal, '3' en Bas (?)


(?) de combien ? (?)
7 |   |   |   |   |   |   |   |   |
6 |   |   |   |   |   |   | o |   |
5 |   |   |   |   |   |   |   |   |
4 |   |   |   |   |   |   |   |   |
3 |   |   |   |   |   |   |   |   |
2 |   |   |   |   |   |   |   |   |
1 |   |   |   |   |   |   |   |   |
0 |   |   |   |   |   |   |   |   |
    0   1   2   3   4   5   6   7
Au tour du joueur 2 
(?) '1' pour déplacement à Gauche, '2' Diagonal, '3' en Bas (?)
(?) de combien ? (?)
7 |   |   |   |   |   |   |   |   |
6 |   |   |   |   |   |   |   |   |
5 |   |   |   |   |   |   |   |   |
4 |   |   |   |   |   |   | o |   |
3 |   |   |   |   |   |   |   |   |
2 |   |   |   |   |   |   |   |   |
1 |   |   |   |   |   |   |   |   |
0 |   |   |   |   |   |   |   |   |
    0   1   2   3   4   5   6   7
Au tour du joueur 1
(?) '1' pour déplacement à Gauche, '2' Diagonal, '3' en Bas (?)


ValueError: invalid literal for int() with base 10: ''